In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import cftime
import datetime
from datetime import date
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy
import pandas
from PIL import Image
import random
import xarray as xr

In [3]:
Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Data/Output_Files_V6/'
Output_Diri = '/glade/u/home/whimkao//ExtraTrack/ExtraTrack_Github/RCP_Figs/Analysis_Figs_V6.7.2/'

In [4]:
# Open File
def Open_File(File):
    DF = pandas.read_csv(File)
    DF = DF.drop("Unnamed: 0", axis=1)
    return (DF)

In [5]:
# Open Each File
def Files_Open(Model, Diri):
    Data_DF = Open_File(Diri+Model+'_Data_hPa_Output_V6.csv')
    ET_DF = Open_File(Diri+Model+'_ET_hPa_Output_V6.csv')
    Codes_DF = Open_File(Diri+Model+'_Codes_Output_V6.csv')
    Time, Begin_Time, Compl_Time, Peak_Time = [], [], [], []
# Edit Time Format
    for i in range(len(Data_DF)):
        Time.append(Datetime(Data_DF["Time(Z)"][i]))
    for j in range(len(ET_DF)):
        Begin_Time.append(Datetime(ET_DF["ET Begin Time"][j]))
        Compl_Time.append(Datetime(ET_DF["ET Complete Time"][j]))
        Peak_Time.append(Datetime(ET_DF["Peak Time"][j]))
    Data_DF["Time(Z)"] = Time
    ET_DF["ET Begin Time"] = Begin_Time
    ET_DF["ET Complete Time"] = Compl_Time
    ET_DF["Peak Time"] = Peak_Time
    return (Data_DF, ET_DF, Codes_DF)

In [6]:
def Datetime(Time):
    New_Time = datetime.datetime.strptime(Time, '%Y-%m-%d %H:%M:%S')
    return (New_Time)

In [7]:
# Find a Specific Storm Within the DataFrame
def Find_Storm(DF, Code):
    DF_Storm = DF[DF["Code"] == Code].reset_index()
    return (DF_Storm)

In [8]:
# Create Bins
def Create_Bins(Min, Max, Bin_Width):
    Bins = numpy.arange(Min, Max+Bin_Width, Bin_Width)
    return (Bins)

In [9]:
Control_Data, Control_ET, Control_Codes = Files_Open("Control", Diri)
RCP45_Data, RCP45_ET, RCP45_Codes = Files_Open("RCP45", Diri)
RCP85_Data, RCP85_ET, RCP85_Codes = Files_Open("RCP85", Diri)

In [10]:
# Function to Find Distance Between Two Points
def Find_Distance(y1, y2, x1, x2):
    Start_Lat = y1 * numpy.pi / 180
    End_Lat = y2 * numpy.pi / 180
    Start_Lon = x1 * numpy.pi / 180
    End_Lon = x2 * numpy.pi / 180
    Lat_Diff = End_Lat - Start_Lat
    Lon_Diff = End_Lon - Start_Lon
    Earth_Rad = 6378
    Distance = 2 * Earth_Rad * numpy.sqrt((numpy.sin(Lat_Diff/2))**2 + \
    numpy.cos(Start_Lat) * numpy.cos(End_Lat) * (numpy.sin(Lon_Diff/2))**2)
    return (Distance)

In [11]:
# Create Function to Open Storm Composite Files
def Composite_File(File):
    Diri = '/glade/campaign/univ/upsu0032/Hyperion_ET/composites/'
    Compo_File = xr.open_dataset(Diri + File)
    return (Compo_File)

In [12]:
# Open Storm Composite Files
Control_A_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
Control_B_Compo_nc = Composite_File('composite_h3_CORI.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
Control_C_Compo_nc = Composite_File('composite_h3_CHEY.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')

In [13]:
# Open Storm Composite Files
RCP45_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP45_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.002.nc')
RCP45_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP45.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')

In [14]:
# Open Storm Composite Files
RCP85_A_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.nc')
RCP85_B_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.003.nc')
RCP85_C_Compo_nc = Composite_File('composite_h3_CHEY.RCP85.VR28.NATL.REF.CAM5.4CLM5.0.dtime900.004.nc')

In [15]:
# Create DataFrame With Lat Lon Time Data of the Composite Files
def Composite_DF(Compo_nc, ABC):
    Snap_Time = pandas.Series(Compo_nc.snap_time)
    Snap_Lon = pandas.Series(Compo_nc.snap_lon)
    Snap_Lat = pandas.Series(Compo_nc.snap_lat)
    Snap_PathID = pandas.Series(Compo_nc.snap_pathid)
    Index = numpy.arange(0,len(Snap_Time),1)
    ABC_List = []
    for m in range(len(Index)):
        ABC_List.append(ABC)
    Compo_DF = pandas.DataFrame({"Orig Index": Index, "ABC": ABC_List, \
    "Time": Snap_Time, "Lon": Snap_Lon, "Lat": Snap_Lat, "PathID": Snap_PathID})
    return (Compo_DF)

In [16]:
# Combine Composite DFs
def Combine_Compo_DF(Compo_A, Compo_B, Compo_C):
    Compo_DF_A = Composite_DF(Compo_A, "A")
    Compo_DF_B = Composite_DF(Compo_B, "B")
    Compo_DF_C = Composite_DF(Compo_C, "C")
    Compo_DF = pandas.concat([Compo_DF_A, Compo_DF_B, Compo_DF_C]).reset_index()
    Compo_DF = Compo_DF.drop("index", axis=1)
    return (Compo_DF)

In [17]:
Control_Compo = Combine_Compo_DF(Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)

In [18]:
RCP45_Compo = Combine_Compo_DF(RCP45_A_Compo_nc, RCP45_B_Compo_nc, RCP45_C_Compo_nc)

In [19]:
RCP85_Compo = Combine_Compo_DF(RCP85_A_Compo_nc, RCP85_B_Compo_nc, RCP85_C_Compo_nc)

In [20]:
Control_Compo[Control_Compo["ABC"] == "A"]

,Orig Index,ABC,Time,Lon,Lat,PathID
0,0,A,1985-06-19 12:00:00,-82.19,27.99,0
1,1,A,1985-06-19 18:00:00,-82.26,27.79,0
2,2,A,1985-06-20 00:00:00,-82.33,27.58,0
3,3,A,1985-06-20 06:00:00,-82.40,27.38,0
4,4,A,1985-06-20 12:00:00,-82.46,28.31,0
...,...,...,...,...,...,...
10154,10154,A,2014-11-11 06:00:00,-76.86,26.37,306
10155,10155,A,2014-11-11 12:00:00,-75.85,27.61,306
10156,10156,A,2014-11-11 18:00:00,-74.54,29.19,306
10157,10157,A,2014-11-12 00:00:00,-73.07,32.06,306


In [21]:
# Change Year of Data
def Reverse_Update_Year(New_Time, Year_Diff):
    Year_Orig = New_Time.year + Year_Diff
#    print (Year_Orig)
    Orig_Time = New_Time.replace(year=Year_Orig)
    return (Orig_Time)

In [22]:
# Create Function to Find Year Diff
def Year_Diff_Find(New_Time):
    Years = [1900,1930,1960,2000,2031,2062,2100,2131,2162,2193]
    New_Time_Index = -728
    for i in range(len(Years)):
        if i < 3:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 1985 - Years[i]
                New_Time_Index = i
        elif i < 6:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
        else:
            if New_Time.year >= Years[i] and New_Time.year < Years[i+1]:
                Year_Diff = 2070 - Years[i]
                New_Time_Index = i
    if New_Time_Index % 3 == 0:
        ABC = "A"
    elif New_Time_Index % 3 == 1:
        ABC = "B"
    elif New_Time_Index % 3 == 2:
        ABC = "C"
    return (int(Year_Diff), ABC)

In [66]:
# Create Function to Find Indexes of Composite Data For Selected Storm
def Find_Composite_Data(Code, Data_DF, Compo_DF):
    DF_Storm = Find_Storm(Data_DF, Code)
    Code_List = DF_Storm["Code"]
    New_Time = DF_Storm["Time(Z)"]
    Lat = DF_Storm["Lat"]
    Lon = DF_Storm["Lon"]
    SLP = DF_Storm["SLP(hPa)"]
#    Windspeed = DF_Storm["Winds(m/s)"]
    Storm_Phase = DF_Storm["Storm Phase"]
    Compo_Indexes = numpy.zeros(len(New_Time))
    for i in range(len(New_Time)):
        Year_Diff, ABC = Year_Diff_Find(New_Time[0])
        Orig_Time = Reverse_Update_Year(New_Time[i], Year_Diff)
# Find Possible Storms that Occur at the Same Time
        Compo_Storm = Compo_DF[(Compo_DF["ABC"] == ABC) & (Compo_DF["Time"] == Orig_Time)].reset_index()
# If No Storm Found:
        if len(Compo_Storm) == 0:
            Compo_Indexes[i] = -728
# Storms Found:
        else:
            Dist_Min = [7428,-728]
            for c in range(len(Compo_Storm)):
                Dist = Find_Distance(Lat[i], Compo_Storm["Lat"][c], Lon[i], Compo_Storm["Lon"][c])
# Find Storm Closest to Storm Center
                if Dist < Dist_Min[0]:
# At Most 300km of Error in Location Permitted
                    if Dist < 300:
                        Dist_Min = [Dist, Compo_Storm["Orig Index"][c]]
                    else:
                        Dist_Min = [Dist, -728]
            Compo_Indexes[i] = Dist_Min[1]
    DF_Storm_Compo_Init = pandas.DataFrame({"Code": Code_List, "Compo Index": Compo_Indexes, "Time": New_Time, \
    "Lon": Lon, "Lat": Lat, "SLP": SLP, "Storm Phase": Storm_Phase})
# Remove Datapoints With Missing Compo Index
    DF_Storm_Compo = DF_Storm_Compo_Init[DF_Storm_Compo_Init["Compo Index"] >= 0].reset_index()
    DF_Storm_Compo = DF_Storm_Compo.drop("index", axis=1)
    return (DF_Storm_Compo)

In [24]:
def Windspeed_850hPa(Compo_nc, Compo_Index):
    U850 = numpy.array(Compo_nc.snap_U850[int(Compo_Index)])
    V850 = numpy.array(Compo_nc.snap_V850[int(Compo_Index)])
    Snap_850 = numpy.sqrt(U850 **2 + V850 **2)
    return (Snap_850)

In [25]:
# Find Total Precip Rate From Compo File
def Precip(Compo_nc, Compo_Index):
    Precip_ms = numpy.array(Compo_nc.snap_PRECT[int(Compo_Index)])
    Precip_mmhr = Precip_ms * 3600 * 1000
    return (Precip_mmhr)

In [44]:
# Find 850hPa Max Windspeed and Wind Field Size at Each 6 Hourly Data Point
def Wind_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    SLP = DF_Storm_Compo["SLP"]
#
# Create Array to Store Data
    Wind_Field_Info = numpy.zeros((5,len(Compo_Index)))
    Wind_Field_Info[0] = SLP
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Windspeed Snap From Compo_nc
        Snap_850 = Windspeed_850hPa(Compo_nc, Compo_Index[k])
# Find Maximum 850hPa Windspeed
        Windspeed_850 = numpy.max(Snap_850)
        Wind_Field_Info[1][k] = Windspeed_850
# Count Number of Data Points With Windspeed Above 18,25,33m/s
        Snap_Sort = numpy.sort(Snap_850.ravel())
        Count_18 = len(Snap_Sort[Snap_Sort >= 18])
        Count_25 = len(Snap_Sort[Snap_Sort >= 25])
        Count_33 = len(Snap_Sort[Snap_Sort >= 33])
        Wind_Field_Info[2][k] = Count_18
        Wind_Field_Info[3][k] = Count_25
        Wind_Field_Info[4][k] = Count_33
#
# Add Wind Field Info Into DF Storm Compo
    DF_Storm_Compo["850hPa Winds"] = Wind_Field_Info[1]
    DF_Storm_Compo["18m/s"] = Wind_Field_Info[2]
    DF_Storm_Compo["25m/s"] = Wind_Field_Info[3]
    DF_Storm_Compo["33m/s"] = Wind_Field_Info[4]
    return (DF_Storm_Compo)

In [31]:
# Find Max Precip and Total Precip Over Area
def Precip_Field_Find(DF_Storm_Compo, Compo_nc):
    Compo_Index = DF_Storm_Compo["Compo Index"]
    Time_List = DF_Storm_Compo["Time"]
#
# Create Array to Store Data
    Precip_Field_Info = numpy.zeros((5,len(Compo_Index)))
#
# At Each 6 Hourly Data Point
    for k in range(len(Compo_Index)):
# Find 850hPa Precip Snap From Compo_nc
        Snap_Precip = Precip(Compo_nc, Compo_Index[k])
# Find Maximum Precip Rate
        Max_Precip = numpy.max(Snap_Precip)
        Precip_Field_Info[0][k] = Max_Precip
# Find Areal Precip Total
        Snap_Sort = numpy.sort(Snap_Precip.ravel())
        Areal_Precip_Total = numpy.sum(Snap_Sort)
        Precip_Field_Info[1][k] = Areal_Precip_Total
# Count Number of Data Points With Precip Rate Above 0.5, 1, 5mm/hr
        Count_05 = len(Snap_Sort[Snap_Sort >= 0.5])
        Count_1 = len(Snap_Sort[Snap_Sort >= 1])
        Count_5 = len(Snap_Sort[Snap_Sort >= 5])
        Precip_Field_Info[2][k] = Count_05
        Precip_Field_Info[3][k] = Count_1
        Precip_Field_Info[4][k] = Count_5
# 
# Add Precip Field Info Into DF Storm Compo
    DF_Storm_Compo["Max Precip Rate"] = Precip_Field_Info[0]
    DF_Storm_Compo["Areal Precip Total"] = Precip_Field_Info[1]
    DF_Storm_Compo["0.5mm/hr"] = Precip_Field_Info[2]
    DF_Storm_Compo["1mm/hr"] = Precip_Field_Info[3]
    DF_Storm_Compo["5mm/hr"] = Precip_Field_Info[4]
    return (DF_Storm_Compo)

In [39]:
# Create New Data DF
def DF_Data_Compo(Data_DF, ET_DF, Compo_DF, Codes_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = Codes_DF["New Code"]
    ABC_List = Codes_DF["ABC"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Composite_Data(Code_List[n], Data_DF, Compo_DF)
# Find Which Compo nc To Use
        if ABC_List[n] == "A":
            Compo_nc = Compo_nc_A
        elif ABC_List[n] == "B":
            Compo_nc = Compo_nc_B
        elif ABC_List[n] == "C":
            Compo_nc = Compo_nc_C
# Apply Functions For Finding Wind Field and Precip Field
        DF_Storm_Compo = Wind_Field_Find(DF_Storm_Compo, Compo_nc)
        DF_Storm_Compo = Precip_Field_Find(DF_Storm_Compo, Compo_nc)
# Combine DF Storm Compos
        if n == 0:
            Data_Compo = DF_Storm_Compo.copy()
        else:
            Data_Compo = pandas.concat([Data_Compo, DF_Storm_Compo])
    Data_Compo_Final = Data_Compo.reset_index().drop("index", axis=1)
    return (Data_Compo_Final)

In [29]:
Control_ET

,Code,Name,ABC,Trans Type,ET Begin Time,ET Complete Time,Peak Time,Peak SLP,Peak Lon,Peak Lat,ET Begin SLP,ET Begin Lon,ET Begin Lat,ET Complete SLP,ET Complete Lon,ET Complete Lat
0,TC190001,Alex,A,3,1900-07-03 18:00:00,1900-07-03 18:00:00,1900-06-25 06:00:00,987.41,-84.91,29.95,1015.57,-90.75,36.25,1015.57,-90.75,36.25
1,TC190003,Colin,A,1,1900-09-18 00:00:00,1900-09-20 12:00:00,1900-09-19 12:00:00,934.83,-70.67,39.42,952.06,-77.17,31.14,979.68,-62.68,47.61
2,TC190004,Dawn,A,1,1900-11-05 12:00:00,1900-11-06 12:00:00,1900-11-04 00:00:00,948.88,-91.58,20.70,954.74,-87.08,27.00,992.74,-74.44,30.88
3,TC190005,Ernest,A,1,1900-11-06 00:00:00,1900-11-08 00:00:00,1900-11-06 18:00:00,988.17,-20.93,46.12,996.02,-27.73,41.42,998.22,-9.00,45.00
4,TC190102,Harrison,A,1,1901-05-28 12:00:00,1901-05-30 18:00:00,1901-05-27 06:00:00,980.89,-51.83,25.92,991.56,-43.71,30.04,1008.78,-34.75,24.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,TC198802,Micah,C,1,1988-02-06 00:00:00,1988-02-06 06:00:00,1988-02-05 06:00:00,998.09,-35.19,36.46,999.63,-31.01,38.29,999.02,-28.74,38.72
285,TC198805,Seldoen,C,1,1988-09-07 06:00:00,1988-09-08 12:00:00,1988-09-07 00:00:00,980.43,-49.24,32.21,984.68,-48.37,33.35,987.91,-35.17,42.45
286,TC198808,Alice,C,2,1988-09-12 06:00:00,1988-09-12 12:00:00,1988-09-13 06:00:00,990.70,-21.25,51.00,1009.08,-39.50,44.00,1008.35,-37.50,45.75
287,TC198809,Bruce,C,1,1988-09-21 06:00:00,1988-09-27 00:00:00,1988-09-22 00:00:00,934.40,-63.31,32.34,961.07,-66.39,30.40,1001.36,-41.00,47.00


In [70]:
# Create New ET DF
def DF_ET_Compo(Data_Compo, ET_DF, Compo_DF, Codes_DF, Compo_nc_A, Compo_nc_B, Compo_nc_C):
    Code_List = ET_DF["Code"]
# Loop Over Each Storm in Dataset
    for n in range(len(Code_List)):
        DF_Storm_Compo = Find_Storm(Data_Compo, Code_List[n])
        ET_Storm = Find_Storm(ET_DF, Code_List[n])
# Find ET Begin and ET Complete Time
        Begin_Time = ET_Storm["ET Begin Time"][0]
        Compl_Time = ET_Storm["ET Complete Time"][0]
        DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
        DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
# Combine ET Storm Compos
        ET_Storm_Compo = Find_ET_Compo(Code_List[n], ET_Storm, DF_Begin, DF_Compl)
        if n == 0:
            ET_Compo = ET_Storm_Compo.copy()
        else:
            ET_Compo = pandas.concat([ET_Compo, ET_Storm_Compo])
    ET_Compo_Final = ET_Compo.reset_index().drop("index", axis=1)
    return (ET_Compo_Final)

In [73]:
def Find_ET_Compo(Code, ET_Storm, DF_Begin, DF_Compl):
    ET_Storm_Compo = ET_Storm.copy()
    Vars = ["850hPa Winds", "18m/s", "25m/s", "33m/s", \
    "Max Precip Rate", "Areal Precip Total", "0.5mm/hr", "1mm/hr", "5mm/hr"]
    print (Code, len(DF_Begin), len(DF_Compl))
    for m in range(len(Vars)):
        Var = Vars[m]
        Begin_Var = str("ET Begin " + Var)
        Compl_Var = str("ET Complete " + Var)
        ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
        ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
    return (ET_Storm_Compo)

In [60]:
DF_Storm_Compo = Find_Storm(Control_Data_Compo, "TC191304")
ET_Storm = Find_Storm(Control_ET, "TC191304")
Begin_Time = ET_Storm["ET Begin Time"][0]
Compl_Time = ET_Storm["ET Complete Time"][0]
DF_Begin = DF_Storm_Compo[DF_Storm_Compo["Time"] == Begin_Time].reset_index()
DF_Compl = DF_Storm_Compo[DF_Storm_Compo["Time"] == Compl_Time].reset_index()
DF_Compl

,level_0,index,Code,Compo Index,Time,Lon,Lat,SLP,Storm Phase,850hPa Winds,18m/s,25m/s,33m/s,Max Precip Rate,Areal Precip Total,0.5mm/hr,1mm/hr,5mm/hr
0,60,1615,TC191304,3999.0,1913-09-08 18:00:00,-51.01,50.86,968.71,Extratropical,40.247997,2478.0,1330.0,324.0,9.796552,1767.507935,778.0,552.0,36.0


In [61]:
ET_Storm_Compo = ET_Storm.copy()
Vars = ["850hPa Winds", "18m/s", "25m/s", "33m/s", \
"Max Precip Rate", "Areal Precip Total", "0.5mm/hr", "1mm/hr", "5mm/hr"]
for m in range(len(Vars)):
    Var = Vars[m]
    Begin_Var = str("ET Begin " + Var)
    Compl_Var = str("ET Complete " + Var)
    ET_Storm_Compo[Begin_Var] = DF_Begin[Var][0]
    ET_Storm_Compo[Compl_Var] = DF_Compl[Var][0]
ET_Storm_Compo

,index,Code,Name,ABC,Trans Type,ET Begin Time,ET Complete Time,Peak Time,Peak SLP,Peak Lon,...,ET Begin Max Precip Rate,ET Complete Max Precip Rate,ET Begin Areal Precip Total,ET Complete Areal Precip Total,ET Begin 0.5mm/hr,ET Complete 0.5mm/hr,ET Begin 1mm/hr,ET Complete 1mm/hr,ET Begin 5mm/hr,ET Complete 5mm/hr
0,40,TC191304,Melody,A,1,1913-09-05 18:00:00,1913-09-08 18:00:00,1913-09-01 18:00:00,894.09,-66.8,...,44.222725,9.796552,2755.665771,1767.507935,962.0,778.0,523.0,552.0,89.0,36.0


In [46]:
Control_Data_Compo = DF_Data_Compo(Control_Data, Control_ET, Control_Compo, Control_Codes, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)
Control_Data_Compo

,Code,Compo Index,Time,Lon,Lat,SLP,Storm Phase,850hPa Winds,18m/s,25m/s,33m/s,Max Precip Rate,Areal Precip Total,0.5mm/hr,1mm/hr,5mm/hr
0,TC190001,0.0,1900-06-19 12:00:00,-82.19,27.99,1015.45,Tropical,23.692062,149.0,0.0,0.0,11.405020,822.940491,407.0,155.0,8.0
1,TC190001,1.0,1900-06-19 18:00:00,-82.26,27.79,1015.45,Tropical,20.203150,69.0,0.0,0.0,12.442551,886.791443,406.0,141.0,3.0
2,TC190001,2.0,1900-06-20 00:00:00,-82.33,27.58,1015.45,Tropical,19.720341,23.0,0.0,0.0,2.922493,760.727722,413.0,83.0,0.0
3,TC190001,3.0,1900-06-20 06:00:00,-82.40,27.38,1015.45,Tropical,20.409258,21.0,0.0,0.0,9.133479,687.665039,404.0,118.0,9.0
4,TC190001,4.0,1900-06-20 12:00:00,-82.46,28.31,1014.22,Tropical,20.912153,13.0,0.0,0.0,11.239372,660.232849,342.0,111.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,TC198902,10447.0,1989-09-28 18:00:00,-5.25,46.00,1016.56,Extratropical,18.245184,1.0,0.0,0.0,7.411370,358.410553,205.0,85.0,3.0
11684,TC198902,10448.0,1989-09-29 00:00:00,-2.50,46.00,1018.02,Extratropical,15.566844,0.0,0.0,0.0,2.564598,201.507980,97.0,28.0,0.0
11685,TC198902,10449.0,1989-09-29 06:00:00,-1.50,46.25,1017.76,Extratropical,12.976286,0.0,0.0,0.0,1.198661,164.178848,110.0,20.0,0.0
11686,TC198902,10450.0,1989-09-29 12:00:00,-0.25,47.00,1018.13,Extratropical,16.618563,0.0,0.0,0.0,2.643814,213.271713,91.0,13.0,0.0


In [75]:
Find_Storm(Control_Data_Compo, "TC190204")

,index,Code,Compo Index,Time,Lon,Lat,SLP,Storm Phase,850hPa Winds,18m/s,25m/s,33m/s,Max Precip Rate,Areal Precip Total,0.5mm/hr,1mm/hr,5mm/hr
0,418,TC190204,622.0,1902-07-15 12:00:00,-88.31,29.21,1012.07,Tropical,18.965036,5.0,0.0,0.0,18.888166,1187.123413,531.0,251.0,40.0
1,419,TC190204,623.0,1902-07-15 18:00:00,-88.55,28.98,1009.72,Tropical,27.605665,35.0,4.0,0.0,14.245308,1483.621948,629.0,268.0,33.0
2,420,TC190204,624.0,1902-07-16 00:00:00,-87.66,28.50,1007.52,Tropical,28.251930,86.0,6.0,0.0,10.871217,1162.428223,561.0,231.0,15.0
3,421,TC190204,625.0,1902-07-16 06:00:00,-86.09,28.18,1006.62,Tropical,29.341576,164.0,14.0,0.0,18.744766,1097.137939,497.0,246.0,31.0
4,422,TC190204,626.0,1902-07-16 12:00:00,-84.32,28.45,1002.21,Tropical,32.167336,229.0,32.0,0.0,20.499496,1424.213989,606.0,294.0,45.0
5,423,TC190204,627.0,1902-07-16 18:00:00,-83.08,28.50,997.71,Tropical,39.635845,328.0,99.0,13.0,35.223988,1790.633545,628.0,346.0,39.0
6,424,TC190204,628.0,1902-07-17 00:00:00,-80.76,28.70,1003.51,Tropical,39.054848,386.0,113.0,32.0,38.561153,1564.967896,500.0,245.0,55.0
7,425,TC190204,629.0,1902-07-17 06:00:00,-79.54,29.11,1003.93,Tropical,42.923351,315.0,135.0,33.0,43.935566,1402.072754,424.0,235.0,59.0
8,426,TC190204,630.0,1902-07-17 12:00:00,-77.58,29.19,1004.12,Tropical,43.093609,324.0,122.0,45.0,18.407524,1147.000366,534.0,292.0,24.0
9,427,TC190204,631.0,1902-07-17 18:00:00,-76.03,29.78,1001.70,Tropical,41.545059,320.0,93.0,31.0,28.247450,1738.176514,611.0,295.0,68.0


In [74]:
Control_ET_Compo = DF_ET_Compo(Control_Data_Compo, Control_ET, Control_Compo, Control_Codes, \
Control_A_Compo_nc, Control_B_Compo_nc, Control_C_Compo_nc)
Control_ET_Compo

TC190001 1 1
TC190003 1 1
TC190004 1 1
TC190005 1 1
TC190102 1 1
TC190105 1 1
TC190106 1 1
TC190107 1 1
TC190109 1 1
TC190202 1 1
TC190203 1 1
TC190204 0 0


KeyError: 0